
# 04 · Umfrage-Reporting (Stichprobe 2025) — Optimiert (v2)

Diese Version behebt einen Fehler bei den Fehlerbalken (Matplotlib: `'xerr' must not contain negative values`).
Die Konfidenzintervalle werden jetzt **aus den tatsächlichen Zählungen** der Stichprobe berechnet und eventuelle
numerische Rundungsartefakte werden **abgesichert (geclippt)**.


In [2]:

# Cell 1: Setup & Daten laden
from __future__ import annotations

from pathlib import Path
import json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

BASE = Path.cwd().resolve().parents[0] if Path.cwd().name.lower()=="notebooks" else Path.cwd()
DATA = BASE / "data"
OUT  = DATA / "processed"
FIG  = (BASE / "reports" / "figures"); FIG.mkdir(parents=True, exist_ok=True)

CONFIG = json.loads((OUT / "project_config.json").read_text(encoding="utf-8"))
KANON  = CONFIG["kanon"]

LONG_PATH = OUT / "umfrage_2025_long.csv"
WIDE_PATH = OUT / "umfrage_2025_wide.csv"
assert LONG_PATH.exists(), f"Fehlt: {LONG_PATH}. Bitte zuerst 04_umfrage_ingest ausführen."
assert WIDE_PATH.exists(), f"Fehlt: {WIDE_PATH}. Bitte zuerst 04_umfrage_ingest ausführen."

df_long = pd.read_csv(LONG_PATH)
df_wide = pd.read_csv(WIDE_PATH).set_index("Kategorie").reindex(KANON).fillna(0.0)

# Respondenten-N: aus Long-Export
# Bei single-choice: Sum(cnt) == N; bei multi-choice: cnt_sum >= N; wir nehmen das Maximum als N.
N_resp = int(df_long["cnt"].max())

# Wide-Prozente robust lesen
def _to_int_if_year(c):
    s = str(c).strip()
    if s.endswith(".0") and s[:-2].isdigit():
        s = s[:-2]
    return int(s) if s.isdigit() else s
df_wide.columns = [_to_int_if_year(c) for c in df_wide.columns]
assert 2025 in df_wide.columns, "Spalte 2025 fehlt in umfrage_2025_wide.csv"

share_2025 = df_wide[2025].rename("share_%").reindex(KANON).fillna(0.0)
df_share = pd.DataFrame({"Kategorie": KANON, "share_%": share_2025.values}).set_index("Kategorie")

# Zählungen je Kategorie für 2025 (tatsächliche k)
cnt_2025 = (df_long.query("year == 2025")[["Kategorie","cnt"]]
                  .set_index("Kategorie").reindex(KANON).fillna(0).astype(int)["cnt"])


In [3]:

# Cell 2: Plot-Helfer (layout/labels)
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 11,
    "axes.titlesize": 12,
    "axes.labelsize": 11,
    "axes.grid": True,
    "grid.alpha": 0.2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "figure.autolayout": False,
})

def percent_axis(ax, axis="x", decimals=0, limit=(0, 100), pad_pct=0):
    lo, hi = limit
    hi_padded = hi + float(pad_pct)
    fmt = mtick.PercentFormatter(xmax=100, decimals=decimals)
    if axis == "x":
        ax.set_xlim(lo, hi_padded)
        ax.xaxis.set_major_formatter(fmt)
    else:
        ax.set_ylim(lo, hi_padded)
        ax.yaxis.set_major_formatter(fmt)

def label_hbars_right(ax, bars, decimals=1, dx=1.0):
    for rect in bars:
        v = rect.get_width()
        ax.text(v + dx, rect.get_y() + rect.get_height()/2,
                f"{v:.{decimals}f} %", va="center", ha="left", fontsize=9, clip_on=False)

def footer(note: str):
    plt.figtext(0.01, -0.04, note, ha="left", va="top", fontsize=9)

def save_fig(path: Path):
    plt.savefig(path, bbox_inches="tight", pad_inches=0.25, facecolor="white")
    plt.close()


In [4]:

# Cell 3: Niveau (horizontal, sortiert) mit Außenlabels
order = df_share["share_%"].sort_values(ascending=True)
fig, ax = plt.subplots(figsize=(10, 6), layout="constrained")
bars = ax.barh(order.index, order.values)
label_hbars_right(ax, bars, decimals=1, dx=1.0)
ax.set_title("Stichprobe 2025: Anteil Respondenten je Kategorie")
ax.set_xlabel("Anteil in %")
percent_axis(ax, axis="x", decimals=0, limit=(0, 100), pad_pct=5)
footer(f"Stichprobe 2025 (N≈{N_resp}). Anteil = Respondenten mit ≥1 Auswahl je Kategorie. Quelle: eigene Umfrage.")
save_fig(FIG / "04_sample_2025_niveau.png")
print("Exportiert →", FIG / "04_sample_2025_niveau.png")


Exportiert → D:\Q3_2025\data-analytics\project\reports\figures\04_sample_2025_niveau.png


In [5]:

# Cell 4: 95%-Konfidenzintervalle (Wilson) mit robustem xerr (nie negativ)
def wilson_ci(k, n, z=1.96):
    if n <= 0:
        return (0.0, 0.0)
    p = k / n
    denom = 1 + z*z/n
    center = (p + z*z/(2*n)) / denom
    half = (z * ((p*(1-p)/n) + (z*z/(4*n*n)))**0.5) / denom
    lo, hi = max(0.0, center - half), min(1.0, center + half)
    return lo, hi

# Vektoriell berechnen
k_vec = cnt_2025.astype(float).values
n = float(N_resp if N_resp else 1.0)
p_vec = k_vec / n
vals = p_vec * 100.0

lo_list, hi_list = [], []
for k in k_vec:
    lo, hi = wilson_ci(k, n, z=1.96)
    lo_list.append(lo*100.0)
    hi_list.append(hi*100.0)
lows  = np.array(lo_list, dtype=float)
highs = np.array(hi_list, dtype=float)

# Ordnung wie im Niveau-Plot
order_idx = df_share["share_%"].sort_values(ascending=True).index
# Reindizieren
vals  = pd.Series(vals, index=cnt_2025.index).reindex(order_idx).values
lows  = pd.Series(lows, index=cnt_2025.index).reindex(order_idx).values
highs = pd.Series(highs, index=cnt_2025.index).reindex(order_idx).values

# xerr als (links, rechts) – niemals negativ, daher clip auf 0
left_err  = np.clip(vals - lows,  0.0, None)
right_err = np.clip(highs - vals, 0.0, None)

y = np.arange(len(order_idx))
fig, ax = plt.subplots(figsize=(10, 6), layout="constrained")
bars = ax.barh(order_idx, vals, zorder=1)
ax.errorbar(x=vals, y=y, xerr=[left_err, right_err], fmt="none", capsize=3, linewidth=1, zorder=2)
label_hbars_right(ax, bars, decimals=1, dx=1.0)

ax.set_title("Stichprobe 2025: Anteil je Kategorie mit 95%-Konfidenzintervallen (Wilson)")
ax.set_xlabel("Anteil in % (mit 95%-KI)")
percent_axis(ax, axis="x", decimals=0, limit=(0, 100), pad_pct=5)
ax.set_ylim(-0.5, len(order_idx)-0.5)

footer(f"Konfidenzintervalle: Wilson, z=1.96. N≈{N_resp}. Anteil = Respondenten mit ≥1 Auswahl je Kategorie.")
save_fig(FIG / "04_sample_2025_niveau_ci.png")
print("Exportiert →", FIG / "04_sample_2025_niveau_ci.png")


Exportiert → D:\Q3_2025\data-analytics\project\reports\figures\04_sample_2025_niveau_ci.png


In [6]:

# Cell 5: Export-Tabelle (Rohwerte + KI)
export = pd.DataFrame({
    "Kategorie": KANON,
    "cnt_2025": cnt_2025.reindex(KANON).values,
    "share_%": df_share.reindex(KANON)["share_%"].values,
})
# KI erneut (für CSV)
def _wilson_row(k):
    lo, hi = wilson_ci(float(k), float(N_resp if N_resp else 1.0), z=1.96)
    return pd.Series({"ci_low_%": lo*100.0, "ci_high_%": hi*100.0})
export = export.join(export["cnt_2025"].apply(_wilson_row))

out_csv = OUT / "umfrage_2025_reporting_table.csv"
export.to_csv(out_csv, index=False, encoding="utf-8")
print("Tabelle exportiert →", out_csv)


Tabelle exportiert → D:\Q3_2025\data-analytics\project\data\processed\umfrage_2025_reporting_table.csv


In [7]:

# 04_umfrage_reporting — NEW Cell: Q5/Q7 laden
import json, numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib.ticker as mtick
change_long_path = OUT / "umfrage_2025_change_long.csv"
assert change_long_path.exists(), f"Fehlt: {change_long_path}. Bitte 04_umfrage_ingest_Q5Q7 ausführen."
df_change_long = pd.read_csv(change_long_path)
meta_path = OUT / "umfrage_2025_meta.json"
meta = json.loads(meta_path.read_text(encoding="utf-8")) if meta_path.exists() else {}
N_total = int(meta.get("respondents", df_change_long.get("N_total", pd.Series([0])).max()))
N_q5 = int(meta.get("q5_N_answered", 0)); N_q7 = int(meta.get("q7_N_answered", 0))
print("N_total =", N_total, "| N(Q5 beantwortet) =", N_q5, "| N(Q7 beantwortet) =", N_q7)

def _percent_axis(ax, axis="x", decimals=0, limit=(0,100), pad=5):
    lo, hi = limit; hi_p = hi + pad
    fmt = mtick.PercentFormatter(xmax=100, decimals=decimals)
    if axis == "x": ax.set_xlim(lo, hi_p); ax.xaxis.set_major_formatter(fmt)
    else: ax.set_ylim(lo, hi_p); ax.yaxis.set_major_formatter(fmt)

def _label_hbars(ax, bars, dx=1.0, fmt="{:.1f}%"):
    for r in bars:
        v = r.get_width()
        ax.text(v+dx, r.get_y()+r.get_height()/2, fmt.format(v), va="center", ha="left", fontsize=9, clip_on=False)

def _footer(fig, text):
    fig.text(0.01, -0.04, text, ha="left", va="top", fontsize=9, color="#555")


N_total = 39 | N(Q5 beantwortet) = 33 | N(Q7 beantwortet) = 16


In [8]:

# Q5 Plot
df_q5 = df_change_long.query("metric=='more_often'").set_index("Kategorie").reindex(KANON).reset_index()
df_q5["share"] = df_q5["more_often_share_resp_%"].fillna(0.0)
order = df_q5.sort_values("share", ascending=True)
fig, ax = plt.subplots(figsize=(11,6), constrained_layout=True)
bars = ax.barh(order["Kategorie"], order["share"], zorder=2)
_label_hbars(ax, bars, dx=1.0)
ax.set_title("Häufiger online als vor fünf Jahren – Anteil Respondenten (2025)")
ax.set_xlabel("Anteil in %"); _percent_axis(ax, axis="x", limit=(0,100), pad=5)
ax.grid(True, linestyle=":", alpha=0.3, zorder=0)
_footer(fig, f"Frage Q5. Nenner = alle Respondenten (N={N_total}); N beantwortet = {N_q5}. Quelle: eigene Umfrage 2025.")
plt.savefig(FIG / "04_q5_more_often.png", bbox_inches="tight", pad_inches=0.25); plt.close()


In [9]:

# Q7 Plot
df_q7 = df_change_long.query("metric=='less_often'").set_index("Kategorie").reindex(KANON).reset_index()
df_q7["share"] = df_q7["less_often_share_resp_%"].fillna(0.0)
order = df_q7.sort_values("share", ascending=True)
fig, ax = plt.subplots(figsize=(11,6), constrained_layout=True)
bars = ax.barh(order["Kategorie"], order["share"], color="#8888ff", zorder=2)
_label_hbars(ax, bars, dx=1.0)
ax.set_title("Seltener online als vor fünf Jahren – Anteil Respondenten (2025)")
ax.set_xlabel("Anteil in %"); _percent_axis(ax, axis="x", limit=(0,100), pad=5)
ax.grid(True, linestyle=":", alpha=0.3, zorder=0)
_footer(fig, f"Frage Q7. Nenner = alle Respondenten (N={N_total}); N beantwortet = {N_q7}. Quelle: eigene Umfrage 2025.")
plt.savefig(FIG / "04_q7_less_often.png", bbox_inches="tight", pad_inches=0.25); plt.close()


In [10]:

# Netto-Delta Plot (pp)
df_net = pd.DataFrame({
    "Kategorie": KANON,
    "more": df_change_long.query("metric=='more_often'").set_index("Kategorie").reindex(KANON)["more_often_share_resp_%"].values,
    "less": df_change_long.query("metric=='less_often'").set_index("Kategorie").reindex(KANON)["less_often_share_resp_%"].values,
})
df_net["delta_pp"] = (df_net["more"] - df_net["less"]).round(1)
order = df_net.sort_values("delta_pp", ascending=True)
fig, ax = plt.subplots(figsize=(11,6), constrained_layout=True)
colors = ["#31c48d" if v>=0 else "#ef4444" for v in order["delta_pp"]]
bars = ax.barh(order["Kategorie"], order["delta_pp"], color=colors, zorder=2)
for r in bars:
    v = r.get_width()
    ax.text(v + (0.6 if v>=0 else -0.6), r.get_y()+r.get_height()/2, f"{v:+.1f} pp",
            va="center", ha="left" if v>=0 else "right", fontsize=9, clip_on=False)
m = max(abs(order["delta_pp"].min()), abs(order["delta_pp"].max()))
ax.set_xlim(-m*1.25, m*1.25)
ax.set_title("Netto-Verschiebung: Häufiger minus Seltener (Prozentpunkte, 2025)")
ax.set_xlabel("Δ in Prozentpunkten (pp)"); ax.grid(True, linestyle=":", alpha=0.3, zorder=0)
_footer(fig, f"Nenner = alle Respondenten (N={N_total}). Positive Werte: häufiger > seltener.")
plt.savefig(FIG / "04_q5q7_netto_delta.png", bbox_inches="tight", pad_inches=0.25); plt.close()
